> 本文档记录学习Scala高级特性时一些关键的知识，如：
1. 模式匹配
2. 类型参数化
3. 抽象成员
4. 隐式转换于隐式参数

参考资料是：[Chisel教程汇总 by _iChthyosaur](https://blog.csdn.net/qq_34291505/article/details/86744581)

# 1. 模式匹配
模式匹配是Scala中一个强大的高级功能，并且在Chisel中被用于硬件的参数化配置，**可以快速地裁剪、配置不同规模的硬件电**

## 样例类: 定义类时，在最前面加上关键`case`
Scala的编译器会自动对样例类添加一些语法便利:
1. 添加一个与类同名的工厂方法, 可以通过“类名(参数)”来构造对象，而不需要“new 类名(参数)
2. 参数列表的每个参数都隐式地获得了一个val前缀
3. 会自动以“自然”的方式实现toString、hashCode和equals方法
4. 添加一个copy方法，用于构造与旧对象只有某些字段不同的新对象，只需通过传入具名参数和缺省参数实现

In [1]:
case class Students(name: String, score: Int)

defined class Students

In [4]:
val stu1 = Students("fj",1)
val stu2 = Students("hmh",2)

// 使用编译器自己创建的copy构造函数
val stuCopy=stu1.copy() 
val stuCopy2=stu1.copy(name="fmt")

stu1: Students = Students("fj", 1)
stu2: Students = Students("hmh", 2)
stuCopy: Students = Students("fj", 1)
stuCopy2: Students = Students("fmt", 1)

## 模式匹配的种类
模式匹配有如下种类：
1. 通配模式
2. 常量模式
3. 变量模式
4. 构造方法模式
5. 序列模式
6. 元组模式
7. 带类型的模式
8. 变量绑定

### 通配模式
使用`_`表示匹配任何对象, `case default`和`case _`等价

In [5]:
def test(x:Any)={
    x match{
        case List(1,2,_) => println("1,2,_")
        case _ => println("not match") // default case
    }
}

test(List(1,2,10))
test(1)
test(List(1,2))

1,2,_
not match
not match


defined function test

### 常量匹配 & 变量匹配
- 常量匹配只能匹配常量本身
- 变量匹配可以匹配任意对象，还可以在后面的语句中使用该变量；**变量模式和通配模式后面不能再跟别的模式了，因为变量模式和通配模式一定会匹配成功**

In [7]:
def test2(x:Any)={
    x match{
        case 1=> println("one") // 常量匹配
        case other=> println("not one, it's: "+other) // 变量匹配
    }
}

test2(1)
test2(List(1,2))

one
not one, it's: List(1, 2)


defined function test2

### 构造模式
模式匹配时匹配**样例类**的构造函数

In [13]:
case class A(param:Int)

def test3(x:Any)={ // x必须声明为Any类型，否则不可能匹配到A(10)
    x match{
        case A(10)=> println("match case class A") // 构造模式匹配
        case _ => println("not match")
    }
}

test3(A(10))
test3(A(1))
test3(0)

match case class A
not match
not match


defined class A
defined function test3

### 序列模式 & 元组模式
- 序列模式匹配Array或者List
- 元组模式匹配Tuple

In [16]:
def test4(x:Any)={
    x match{
        case Array(1,_*) => println("match: "+x) // 序列模式
        case _ => println("mismatch: "+x)
    }
}

test4(Array(1,2,3,4,5))
test4("Hello")

match: [I@9acc9a2
mismatch: Hello


defined function test4

In [19]:
def test5(x:Any)={
    x match{
        case (1,e,"OK") => println("match: "+e) // 元组匹配
        case _ => println("mismatch")
    }
}

test5((1,"hello","OK"))
test5(12)

match: hello
mismatch


defined function test5

### 带类型的模式
模式定义时，也可以声明具体的数据类型。用带类型的模式可以代替类型测试和类型转换

In [22]:
def test6(x:Any)={
    x match{
        case s:String=> s.length // 带类型的匹配
        case m:Map[_,_]=> m.size
        case _ => println("mismatch")
    }
}

test6("Hello")
test6(Map((1,"apple"),(2,"banana")))

defined function test6
res21_1: AnyVal = 5
res21_2: AnyVal = 2

### 变量绑定
对除了变量模式的其他匹配模式，使用`@`做变量绑定

In [23]:
def test7(x:Any)={
    x match{
        case (1,2,e @ 3)=> e // 将3绑定到变量e, 使用e可以返回3
        case _ => println("mismatch")
    }
}

test7((1,2,3))
test7(0)

mismatch


defined function test7
res22_1: AnyVal = 3
res22_2: AnyVal = ()

## 守卫模式

在模式后面，增加`if`判断，在跟`=>`指明对应的动作

In [24]:
def test8(x:Any)={
    x match{
        case i: Int if i>0 => println("positive: "+i) // 守护模式 if i>0, 只有i为正数的时候才匹配成功
        case _ => println("mismatch")
    }
}

test8(2)
test8(0)

positive: 2
mismatch


defined function test8

## 可选值Option
- Some
- None

**TBD**

## 偏函数
偏函数的作用在于划分一个输入参数的可行域，在可行域内对入参执行一种操作，在可行域之外对入参执行其他操作

偏函数有两个抽象方法需要实现：apply和isDefinedAt：
1. `isDefinedAt`用于**判断入参是否在可行域内**，是的话就返回true，否则返回false
2. `apply`是偏函数的函数体，用于对入参执行操作

In [30]:
// 创建一个偏函数isInt
def isInt = new PartialFunction[Any, String]{
    def apply(x:Any)= x.asInstanceOf[Int]+ " is a Int"
    def isDefinedAt(x: Any)= x.isInstanceOf[Int]
}

isInt(1) // 先调用isDefinedAt, 再调用apply函数
isInt.isDefinedAt(1)
isInt.isDefinedAt("1")
isInt.apply(1)

defined function isInt
res29_1: String = "1 is a Int"
res29_2: Boolean = true
res29_3: Boolean = false
res29_4: String = "1 is a Int"

# 参数类型化
通过使用泛型来提高代码的复用率

**TBD**

## 类构造函数
有点像C++里面的模板, 必须声明为抽象类

In [32]:
abstract class A[T]{ // 定义一个抽象类
    val a:T
}

def doesCompile(x: A[Int]){
    println(x)
}



defined class A
defined function doesCompile

# 3. 抽象成员

Scala有四种抽象成员：
1. 抽象val字段
2. 抽象var字段
3. 抽象方法
4. 抽象类型



In [33]:
trait Abstract{
    type T
    def transform(x:T):T
    val initial: T
    var current: T
}

defined trait Abstract

In [34]:
// 继承抽象类, 将T换成具体的String类型
class Concrete extends Abstract{
    type T= String
    def transform(x: String)= x+x
    val initial: String="Hello"
    var current: String="World"
}

defined class Concrete